<a href="https://colab.research.google.com/github/MaxCojocari/CyberPy_bot/blob/main/app/launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=d472315900844e8b5f0cc4c57eb70f42e4a7e34193baeb07ceb64de5a675969b
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [ ]:
from torchtext.vocab import GloVe
glove = GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399999/400000 [00:09<00:00, 43747.72it/s]


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import torch
from torchtext.data.utils import get_tokenizer
from torch import nn
from torchtext.vocab import GloVe

import string


#nltk.download('stopwords')
from nltk.corpus import stopwords
list_stopwords = set(stopwords.words('english'))

tokenizer = get_tokenizer('basic_english')

glove = GloVe(name='6B', dim=50)

def get_tokens(text):
    tokens = [word for word in tokenizer(text) if word not in list_stopwords]
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens]
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


class LSTM(nn.Module):
    #define all the layers used in model
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional=True, dropout=0.2):
        
        #Constructor
        super().__init__()          
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, embeds):
        packed_output, (hidden, cell) = self.lstm(embeds)
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2, : , : ], hidden[-1, : , : ]), dim = 1)
        #hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)

        return dense_outputs


device = torch.device('cpu')

label2intent_1 = {0: 'not_cyberbullying', 1: 'gender', 2: 'religion', 3: 'age', 4: 'ethnicity'}
label2intent_2 = {0: 'ham', 1: 'spam'}
model1 = LSTM(50, 32, len(label2intent_1), 2).to(device)
model2 = LSTM(50, 32, len(label2intent_2), 2).to(device)
model1.load_state_dict(torch.load(
    '/content/drive/MyDrive/NLPC2022/cyberbullying_lstm.pt', map_location='cpu'))
model2.load_state_dict(torch.load(
    '/content/drive/MyDrive/NLPC2022/spam_lstm.pt', map_location='cpu'))
model1.eval()
model2.eval()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def index():
    # request data from client
    data = request.json
    
    # generate embeds for set of tokens
    embeds = glove.get_vecs_by_tokens(get_tokens(data["text"]))
    embeds = embeds.view(1, embeds.shape[0], 50).to(device)
    
    # give predictions
    output1, output2 = model1(embeds), model2(embeds)

    # find predicted label
    label_cyb = int(torch.max(output1, dim=1)[1][0])
    label_spam = int(torch.max(output2, dim=1)[1][0])

    # softmax perquisite
    m = nn.Softmax(dim=1)
    prob = m(output1)
    
    prediction1, prediction2 = 0, 0
    
    if label_cyb in [1, 2, 3, 4]:
        prediction1 = 1
    
    prediction2 = int(label_spam)
    
    return jsonify({
        "cyberbullying" : prediction1,
        "spam" :  prediction2,
        "cyberbull_type" : {
            'not_cyberbullying': round(prob[0][0].item(), 4),
            'gender': round(prob[0][1].item(), 4),
            'religion': round(prob[0][2].item(), 4),
            'age': round(prob[0][3].item(), 4),
            'ethnicity': round(prob[0][4].item(), 4)
        }
    })

run_with_ngrok(app)
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9865-34-80-27-204.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/May/2022 10:43:04] "GET / HTTP/1.1" 200 -
